In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM

In [4]:
import json
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [5]:
f=open('words.txt','r')
words=f.read().split('\n')
words

['Boy',
 'Can',
 'Eat',
 'Fine',
 'Girl',
 'Help',
 'How',
 'Hungry',
 'I',
 'Mother',
 'Namaste',
 'Name',
 'Parents',
 'Sister',
 'Sleep',
 'This',
 'You',
 '']

In [1]:
X=pd.read_csv('landmarks.csv')
X.head()

,Unnamed: 0,Landmarks
0,0,[[-0.2904878 -0.16641041 -0.09852826 ... -0.3...
1,1,[[-1.14277988 -1.14277988 -1.14277988 ... -0.5...
2,2,[[-1.06837073 -1.06837073 -1.06837073 ... -0.4...
3,3,[[-0.22918894 -0.11131902 -0.05098939 ... -0.7...
4,4,[[-0.87611416 -0.80004271 -0.7496262 ... -0.3...


In [ ]:
Y=pd.read_csv('labels.csv')
Y.head()

In [ ]:
Y=to_categorical(Y).astype(int)
Y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Model-1

In [ ]:
model=Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(62, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[tb_callback])

In [ ]:
model.summary()

MODEL 2

In [ ]:
model=Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [ ]:
res=model.predict(x_test)
res[0]

##Save model

In [ ]:
model.save('model.h5')

model=tf.keras.models.load_model('model.h5')

Prediction

In [ ]:
words[np.argmax(model.predict(x_test)[0])]

In [ ]:
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
y_pred=model.predict(x_test)

In [ ]:
y_true=np.argmax(y_test, axis=1).tolist()
y_pred=np.argmax(y_pred, axis=1).tolist()

In [ ]:
y_true

In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true, y_pred)